In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
df=pd.read_csv('train.csv')
print(df)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [3]:
from sklearn.preprocessing import OneHotEncoder

In [5]:
def data_cleaning (X:pd.DataFrame):

    encoder = OneHotEncoder(sparse_output=False,categories="auto")
    new_column = pd.DataFrame(encoder.fit_transform(X[['Sex']]),columns=['female','male'])
    #print(new_colunm)
    X=X.drop(['Sex'],axis=1)
    X=pd.concat([X,new_column],axis=1)

    
    new_column = pd.DataFrame(encoder.fit_transform(X[['Embarked']]))
    column_name = encoder.get_feature_names_out(['Embarked'])
    new_column.columns=column_name
    X=X.drop(['Embarked'],axis=1)
    X=pd.concat([X,new_column],axis=1)
    
    X['Age']=X['Age'].fillna(X['Age'].mean())

    X=X.drop(['Embarked_nan','Cabin','Name','Ticket'],axis=1)
    return X



In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

In [7]:
train = data_cleaning(df)
split = StratifiedShuffleSplit(n_splits=1,test_size=0.1)
for train_index,test_index in split.split(train,train[['Survived','Pclass','male','female']]):
    train_date = train.loc[train_index]
    test_date = train.loc[test_index]
print(train_date)

     PassengerId  Survived  Pclass        Age  SibSp  Parch      Fare  female  \
434          435         0       1  50.000000      1      0   55.9000     0.0   
870          871         0       3  26.000000      0      0    7.8958     0.0   
222          223         0       3  51.000000      0      0    8.0500     0.0   
60            61         0       3  22.000000      0      0    7.2292     0.0   
792          793         0       3  29.699118      8      2   69.5500     1.0   
..           ...       ...     ...        ...    ...    ...       ...     ...   
1              2         1       1  38.000000      1      0   71.2833     1.0   
734          735         0       2  23.000000      0      0   13.0000     0.0   
652          653         0       3  21.000000      0      0    8.4333     0.0   
551          552         0       2  27.000000      0      0   26.0000     0.0   
337          338         1       1  41.000000      0      0  134.5000     1.0   

     male  Embarked_C  Emba

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [9]:
train_date_X=train_date.drop(['Survived'],axis=1)
train_date_y=train_date['Survived'].to_numpy()

scaler = StandardScaler() #数据中心化，均值0，方差1
train_date_X = scaler.fit_transform(train_date_X)

test_date_X=test_date.drop(['Survived'],axis=1)
test_date_y=test_date['Survived'].to_numpy()

scaler = StandardScaler()
test_date_X = scaler.fit_transform(test_date_X)



In [10]:
clf = RandomForestClassifier()

clf.fit(train_date_X,train_date_y)
 
param_grid = [{'n_estimator':[10,100,200,500]}]

In [11]:
y_pred=clf.predict(test_date_X)
score = clf.score(test_date_X,test_date_y)
print(score)

0.7888888888888889
